## Преобразование текстового документа с карточками таксации в таблицу в формате csv

Проходим построчно по документу data.txt.
Если строка состоит из одних пробелов, то в список добавляется строка нулевой длины (''), а если нет - то полная строка без знака переноса ('\n')

In [1]:
with open('data.txt') as f:
    first_list = f.readlines()

initial_list = []
for i in first_list:
    if i.isspace():
        initial_list.append(i.strip())
    else:
        initial_list.append(i.rstrip('\n'))

Проходим по строкам, находя строки вида "Квартал :   1", чтобы записать первое упоминание каждого из кварталов. Эти номера строк образуют границы, между которыми выдела будут принадлежать только определенному кварталу:

In [2]:
kv_dict = {}

for n, i in enumerate(initial_list):  #получаем с каких строк начинаются определенные квартала
    if 'Квартал' in i:
        kv = int(i.split()[-1])
        if kv not in kv_dict:
            kv_dict[kv] = n

Проверяем количество кварталов:

In [3]:
print(f'{len(kv_dict)} кварталов')

89 кварталов


Словарь не совсем удобный для наших целей формат данных (неудобно итерировать), преобразуем его в два списка одинаковой длины - список первых строк кварталов и список номеров кварталов. Два элемента с одинаковым индексом из этих двух списков будут соотвествовать паре "номер квартала - первая строка" из словаря. Отдельный список номеров кварталов необходим также потому что номера кварталов не идут подряд - так, всего кварталов 89, а их номера доходят до 206.

In [4]:
list_of_kv_str = [k for v, k in kv_dict.items()]
list_of_kv_numbers = [v for v, k in kv_dict.items()]

In [5]:
print(f'Длина списка первых строк - {len(list_of_kv_str)}')   
print(f'Длина списка номеров кварталов - {len(list_of_kv_numbers)}')

Длина списка первых строк - 89
Длина списка номеров кварталов - 89


Отдельные выдела можно отличить тем, что в каждом из них есть строка вида "2005 2Олч -100 30 28 30  5    С2  109   22 1", т.е. во всех выделах встречается подстрока "2005" - она находится либо на третьей, либо на десятой строке выдела (в случаях когда выдел находится на двух страницах документа и разорван "шапкой" таблицы). В этих случаях мы должны проверить, что первый элемент j-2 (j - строка с подстрокой "2005") числовой с помощью функции isdigit()

In [6]:
indexes_of_dates = []

for n, i in enumerate(initial_list):
    if '2005' in i:
        indexes_of_dates.append(n)

Посмотрим на следующий выдел:

"5 4Е   -140  1 29 32  7  2 0.5  28   21 1   4    4          
   2.0 2Е   - 90 -- 25 24 --    КС  ---   11 1 ---  ---          
  2005 3Б   -140 28 27 26  3    В2   55   17 1  -    -           
       1Ос  -140    30 40                  6 4                   
       Подрост: 10Е 20 лет, высота 3.0 м, 2.0 тыс.шт/га       "

В иных случаях подстрока "2005" находится не на третьей строке, а на десятоЙ. 

В данной ситуации нам необходимо записать только первые четыре строки. В данной ситуации можно завершить цикл записи из строк в словарь, встретив подстроку "Подрост". В иных выделах встречаются другие подстроки. Составим список этих слов:

In [7]:
banned_words = ['ОЗУ', '---', 'Подрост', 'Единичные', 'Целевая', 'Состояние', '2-ой ярус', 'Ест', "Тип", 'ширина', 'Подлесок', 'Выполнено', 'Характеристика', 'Потери', 'Ландшафтная', 'Полнота', "Состав", 'лесосеменная', "прививочная", "Вырубка", "Проек", "Заросшие", "Насаждение", "РГП", "Погиб", "муравейник", "полог", "выдел", "Почвы", '+']

Составим словарь из элементов по три или более строки, где ключом будет являться число вида (пример) "11023", где "11" будет номером квартала, а "23" - номером выдела.


In [8]:
dict_of_strings = {}   

# Пройдем циклом по строкам, где есть подстрока "2005". Это либо третья, либо десятая строка в выделе:
for j in indexes_of_dates:
    str1 = initial_list[j - 2]
    str2 = initial_list[j - 1]

    if str1.split()[0].isdigit():
        # Выделяем в переменную номер выдела:
        num_vyd = int(str1.split()[0]) 
    else:
        str1 = initial_list[j - 9]
        str2 = initial_list[j - 8]
        num_vyd = int(str1.split()[0]) 
    
        # Выделяем в переменную номер квартала:
    num_kv = 0
    for l, m in enumerate(list_of_kv_str):
        if j > m:
            num_kv = list_of_kv_numbers[l]
            pass
    
    str3 = initial_list[j] 

    little_list = [str1, str2, str3]

    start = j + 1 # Заводим переменную-счетчик, первое значение - строка за строкой j
    while len(initial_list[start]) != 0: # Проверяем что эта строка start не нулевой длины (поскольку пробелы были обрезаны, то строки, состоящие только из пробелов, стали нулевой длины)
        if any(x in initial_list[start] for x in banned_words):  # Если какие-то из запрещенным подстрок содержатся в строке - 
            break                                                # цикл прерывается
        else:
            little_list.append(initial_list[start]) # Если правило запрещенного слова не нарушено, то строка записывается в список и
            start += 1                              # переменная start растет на единицу и цикл переходит на следующую строку
    
    dict_of_strings[1000 * num_kv + num_vyd] = little_list

Проверим, что длина получившегося словаря равна количеству выделов:

In [9]:
print(f'Длина словаря - {len(dict_of_strings)}')
print(f'Количество выделов - {len(indexes_of_dates)}')

Длина словаря - 3929
Количество выделов - 3929


Проверим, что все строки, записанные в словарь одной длины. Сначала измерим длину одной рандомной строки из словаря.

In [10]:
print(len(dict_of_strings[1001][0]))

65


In [11]:
len_str = 0 
for i in dict_of_strings:
    for j in dict_of_strings[i]:
        if len(j) != 65:   
            len_str += 1   
len_str

0

Как мы убедились, все строки в выделах одинаковой длины. Можно заметить, что все характеристики находятся на определенных столбцах текстового документа - так, например ярус находится в первой строке в диапазоне стобцов с 18 по 20, средняя высота яруса - в третьей строке в диапазоне стобцов с 18 по 20; не общие характеристики выдела, соответствующие определенным ярусам - высота яруса, состав, диаметры деревьев - уже уникальны и находятся в одинаковых стобцах разных строк. Поэтому в функции сначала записываются общие переменные, а потом идет построчный цикл для записи переменных из отдельных ярусов. 
Заводим второй список запрещенных слов для прерывания циклов.

In [12]:
banned_words_2 = ['+', 'км']

In [13]:
def create_row(x):
    number_of_strings = len(dict_of_strings[x])

    # Общие переменные для всего выдела:
    KW = int(x / 1000)
    VD = int(x % 1000)
    if any(t in dict_of_strings[x][0] for t in banned_words_2):
        Tier = Avg_tier_heigth = Age_class = Age_group = Bon = Vyd_reserve = Full = ''
    else:
        Tier = dict_of_strings[x][0][17:20].strip()
        Avg_tier_heigth = dict_of_strings[x][2][17:20].strip()
        Age_class = dict_of_strings[x][0][26:29].strip()
        Age_group =  dict_of_strings[x][2][26:29].strip()
        Bon =  dict_of_strings[x][0][29:32].strip()
        Vyd_reserve =  dict_of_strings[x][2][36:41].strip()
        Full = dict_of_strings[x][0][31:36].strip()

    # Построчные переменные для отдельных строк:
    PR = [''] * 6
    Vys = [''] * 6
    DM = [''] * 6

    for i in range(number_of_strings):
        if any(t in dict_of_strings[x][i] for t in banned_words_2):
            break
        else:
            PR[i] += dict_of_strings[x][i][7:11].strip()
            Vys[i] += dict_of_strings[x][i][20:23].strip()
            DM[i] +=  dict_of_strings[x][i][23:26].strip()

    return [KW, VD, PR[0], PR[1], PR[2], PR[3], PR[4], PR[5], Tier, Avg_tier_heigth, Vys[0], Vys[1], Vys[2], Vys[3], Vys[4], Vys[5], DM[0], DM[1], DM[2], DM[3], DM[4], DM[5], Age_class, Age_group, Bon, Vyd_reserve, Full] 
       

    


Импортируем модуль pandas для работы с табличными данными:

In [14]:
import pandas as pd

Создадим пустой датафрейм с колонками, соотвествующими нижеописанному списку:

In [15]:
columns = ['KW', 'VD', 'PR1', 'PR2', 'PR3', 'PR4', 'PR5', 'PR6', 'Tier', 'Avg_tier_height', 
'Vys_1', 'Vys_2', 'Vys_3', 'Vys_4', 'Vys_5', 'Vys_6', 'DM1', 'DM2', 'DM3', 'DM4', 'DM5', 'DM6', 'Age_class', 'Age_group', 'Bon', 'Vyd_reserve', 'Full']
table = pd.DataFrame(columns=columns)

In [16]:
table

,KW,VD,PR1,PR2,PR3,PR4,PR5,PR6,Tier,Avg_tier_height,...,DM2,DM3,DM4,DM5,DM6,Age_class,Age_group,Bon,Vyd_reserve,Full


Пройдем циклом по элементам словаря, запуская выше инициированную функцию create_row и добавляя ее результат в виде строки к датафрейму table:

In [17]:
for i in list(dict_of_strings):
    table.loc[len(table)] = create_row(i)

In [18]:
table[:10]

,KW,VD,PR1,PR2,PR3,PR4,PR5,PR6,Tier,Avg_tier_height,...,DM2,DM3,DM4,DM5,DM6,Age_class,Age_group,Bon,Vyd_reserve,Full
0,1,1,,,,,,,,,...,,,,,,,,,,
1,1,2,5Е,3Ос,2Б,,,,1,29,...,40,28,,,,5,2,1,23,0.5
2,1,3,4Ос,2Б,2Олч,2Е,,,1,30,...,28,30,28,,,10,5,1А,109,0.6
3,1,4,7Ос,3Б,,,,,1,31,...,26,,,,,8,4,1А,119,0.8
4,1,5,4Е,2Е,3Б,1Ос,,,1,28,...,24,26,40,,,7,3,2,55,0.5
5,1,6,3Е,2Е,3Б,2Ос,,,1,26,...,28,24,32,,,5,2,2,27,0.3
6,1,7,3Е,2Е,4Б,1Ос,,,1,25,...,28,24,32,,,5,2,2,13,0.3
7,1,8,4Е,2Е,3Б,1Ос,,,1,27,...,24,26,36,,,7,3,2,77,0.5
8,1,9,5Е,2Е,2Б,1Ос,,,1,26,...,24,22,30,,,7,3,2,32,0.7
9,1,10,6Е,2Б,2Ос,,,,1,24,...,20,28,,,,3,2,1,41,1.0


Убедимся в том, что число строк в таблице table соответствует количеству выделов (3929):

In [19]:
table.shape

(3929, 27)

Запишем датафрейм в таблицу final_table.csv:

In [20]:
table.to_csv('final_table.csv', sep='\t', encoding='utf-8')